Set the filename

In [1]:
# filename = "/home/gregorylemieux/Work/Issues/383/deciduous-griddedruns/cesm.log.1805694.chadmin1.ib0.cheyenne.ucar.edu.200417-103849"
filename = "/home/gregorylemieux/Work/Issues/383/deciduous-griddedruns/cesm.log.1851933.chadmin1.ib0.cheyenne.ucar.edu.200420-102252"

"/home/gregorylemieux/Work/Issues/383/deciduous-griddedruns/cesm.log.1851933.chadmin1.ib0.cheyenne.ucar.edu.200420-102252"

In [2]:
# Open the file
f = open(filename)

# Read through each line and pull out the unique threadnumbers
A = Array{Int64}(undef,0) # Initialize
@time for line in eachline(f)
    threadnum = parse(Float64,split(line,":")[1]) # Parse the line for the thread number
    if !(in(threadnum,A)) # If not in the array add it
        push!(A,threadnum)
    end
end

# Close the file
close(f)

  1.007146 seconds (12.11 M allocations: 471.641 MiB, 13.45% gc time)


Run a couple of checks

In [3]:
# Check all entries are unique
println(A == unique(A))

# Check that the thread numbers are sequential
println(sort!(A) == Array(0:maximum(A)))

true
true


Run through the file again and count the number of times that the laimemory is zero

In [4]:
# Set the known strings we are interested in
string_canopy = "Starting canopy trim"
string_cohort = "Current cohort"
string_laimem = "Starting laimemory"
string_season = "EDPftvarcon_inst%season_decid"
string_stress = "EDPftvarcon_inst%stress_decid"
string_evergreen = "EDPftvarcon_inst%evergreen"
string_optimum = "OPTIMUM_LAIMEM"
string_cumulative = "CUMULATIVE LAI"
string_trimopt = "OPTIMUM TRIM"

# Create a dictionary to store the information
based = Dict("season"=>0,"stress"=>0,"evergreen"=>0)
znzd = Dict("zero" => copy(based), "nonzero" => copy(based))
outputd = Dict("LAI memory"=>deepcopy(znzd), "LAI optimum" => deepcopy(znzd), "LAI cumulative" => deepcopy(znzd),
               "Trim optimum" => Dict("negative" => copy(based), "zero" => copy(based), "positive" => copy(based)))

totallaicount = 0;

In [8]:
for t in A
#     println(t)    
    laiflag = undef
    seasonflag = 0
    stressflag = 0
    evergreenflag = 0
    
    f = open(filename) # Open the file`
    
    # Iterate through file
    for line in eachline(f)
#         totallinecount += 1
#         outputd["LAI memory"]["zero"]["season"] += 1
        linesplit = split(line,":") # Split into array of strings
        threadnum = parse(Float64,linesplit[1]) # Parse the line for the thread number
        
        # Make sure the line format is threadnum:string:val
        if threadnum == t && length(linesplit) == 3 && isempty(findall(isempty,linesplit))
            if strip(linesplit[2]) == string_laimem
                totallaicount += 1  # Count up the total laimemory loops
                seasonflag = 0
                stressflag = 0
                evergreenflag = 0
                if parse(Float64,strip(linesplit[3])) == 0 # Set the laiflag for the next read
                    laiflag = 0
                else
                    laiflag = 1
                end
            elseif strip(linesplit[2]) == string_season && parse(Float64,strip(linesplit[3])) == 1
                if laiflag == 0
                    outputd["LAI memory"]["zero"]["season"] += 1
                else 
                    outputd["LAI memory"]["nonzero"]["season"] += 1                
                end
                laiflag = undef
                seasonflag = 1
            elseif strip(linesplit[2]) == string_stress && parse(Float64,strip(linesplit[3])) == 1
                if laiflag == 0
                    outputd["LAI memory"]["zero"]["stress"] += 1
                else 
                    outputd["LAI memory"]["nonzero"]["stress"] += 1                
                end
                laiflag = undef
                stressflag = 1
            elseif strip(linesplit[2]) == string_evergreen && parse(Float64,strip(linesplit[3])) == 1
                if laiflag == 0
                    outputd["LAI memory"]["zero"]["evergreen"] += 1
                else 
                    outputd["LAI memory"]["nonzero"]["evergreen"] += 1                
                end
                laiflag = undef
                evergreenflag = 1
            elseif strip(linesplit[2]) == string_optimum
                if parse(Float64,strip(linesplit[3])) == 0
                    if stressflag == 1
                        outputd["LAI optimum"]["zero"]["stress"] += 1
                    elseif seasonflag == 1
                        outputd["LAI optimum"]["zero"]["season"] += 1
                    elseif evergreenflag == 1
                        outputd["LAI optimum"]["zero"]["evergreen"] += 1
                    end
                else
                    if stressflag == 1
                        outputd["LAI optimum"]["nonzero"]["stress"] += 1
                    elseif seasonflag == 1
                        outputd["LAI optimum"]["nonzero"]["season"] += 1
                    elseif evergreenflag == 1
                        outputd["LAI optimum"]["nonzero"]["evergreen"] += 1
                    end
                end
            elseif strip(linesplit[2]) == string_cumulative
                if parse(Float64,strip(linesplit[3])) == 0
                    if stressflag == 1
                        outputd["LAI cumulative"]["zero"]["stress"] += 1
                    elseif seasonflag == 1
                        outputd["LAI cumulative"]["zero"]["season"] += 1
                    elseif evergreenflag == 1
                        outputd["LAI cumulative"]["zero"]["evergreen"] += 1
                    end
                else
                    if stressflag == 1
                        outputd["LAI cumulative"]["nonzero"]["stress"] += 1
                    elseif seasonflag == 1
                        outputd["LAI cumulative"]["nonzero"]["season"] += 1
                    elseif evergreenflag == 1
                        outputd["LAI cumulative"]["nonzero"]["evergreen"] += 1
                    end
                end
            elseif strip(linesplit[2]) == string_trimopt
                if parse(Float64,strip(linesplit[3])) == 0
                    if stressflag == 1
                        outputd["Trim optimum"]["zero"]["stress"] += 1
                    elseif seasonflag == 1
                        outputd["Trim optimum"]["zero"]["season"] += 1
                    elseif evergreenflag == 1
                        outputd["Trim optimum"]["zero"]["evergreen"] += 1
                    end
                elseif parse(Float64,strip(linesplit[3])) > 0
                    if stressflag == 1
                        outputd["Trim optimum"]["positive"]["stress"] += 1
                    elseif seasonflag == 1
                        outputd["Trim optimum"]["positive"]["season"] += 1
                    elseif evergreenflag == 1
                        outputd["Trim optimum"]["positive"]["evergreen"] += 1
                    end
                elseif parse(Float64,strip(linesplit[3])) < 0
                    if stressflag == 1
                        outputd["Trim optimum"]["negative"]["stress"] += 1
                    elseif seasonflag == 1
                        outputd["Trim optimum"]["negative"]["season"] += 1
                    elseif evergreenflag == 1
                        outputd["Trim optimum"]["negative"]["evergreen"] += 1
                    end
                end
            end
        end
    end
close(f) # Close the file
end

In [9]:
outputd["LAI memory"]["nonzero"]

Dict{String,Int64} with 3 entries:
  "stress"    => 6363
  "season"    => 35933
  "evergreen" => 0

In [10]:
outputd["LAI memory"]["zero"]

Dict{String,Int64} with 3 entries:
  "stress"    => 58946
  "season"    => 19355
  "evergreen" => 39700

In [11]:
outputd["LAI optimum"]["nonzero"]

Dict{String,Int64} with 3 entries:
  "stress"    => 0
  "season"    => 0
  "evergreen" => 0

In [12]:
outputd["LAI optimum"]["zero"]

Dict{String,Int64} with 3 entries:
  "stress"    => 20587
  "season"    => 6619
  "evergreen" => 0

In [13]:
outputd["LAI cumulative"]["nonzero"]

Dict{String,Int64} with 3 entries:
  "stress"    => 23878
  "season"    => 8404
  "evergreen" => 1291

In [14]:
outputd["LAI cumulative"]["zero"]

Dict{String,Int64} with 3 entries:
  "stress"    => 0
  "season"    => 0
  "evergreen" => 0

In [15]:
outputd["Trim optimum"]["positive"]

Dict{String,Int64} with 3 entries:
  "stress"    => 21767
  "season"    => 7454
  "evergreen" => 1274

In [16]:
outputd["Trim optimum"]["zero"]

Dict{String,Int64} with 3 entries:
  "stress"    => 0
  "season"    => 0
  "evergreen" => 0

In [17]:
outputd["Trim optimum"]["negative"]

Dict{String,Int64} with 3 entries:
  "stress"    => 2111
  "season"    => 950
  "evergreen" => 17

In [18]:
totallaicount

160297